In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 0. 환경변수 및 패키지

In [1]:
from langchain_core.documents import Document

# pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone.exceptions import NotFoundException
from langchain_upstage import UpstageEmbeddings

# llm
import ollama
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_upstage import ChatUpstage

# 함수
from typing import List, Optional, Dict, Any
from __future__ import annotations
from collections import defaultdict
import re
import json

# api key
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_LLM_MODEL = 'gpt-4o-mini'
OPENAI_EMBEDDIGN_MODEL = 'text-embedding-3-large' 
UPSTAGE_EMBEDDIGN_MODEL = 'solar-embedding-1-large-passage'

PINECONE_INDEX_NAME_LAW = 'law-index-final'
PINECONE_INDEX_NAME_RULE = 'rule-index-final'
PINECONE_INDEX_NAME_CASE = 'case-index-final'

COHERE_API_KEY = os.getenv('COHERE_API_KEY')

# 1. 함수

## 카테고리

In [3]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 통합 카테고리-키워드 가중치 사전
    법령, 판례, 상담 사례를 아우르는 포괄적 검색을 지원합니다.
    """
    return {
        # 1. 보증금_대항력: 보증금 보호 및 우선변제권 (최고 중요 리스크)
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2, '전세보증금': 2,
            '우선매수권': 3, '피해자결정': 2, '경매유예': 3,
            '임대보증금보증': 3, '일부보증': 2, '보증미가입': 3, '동의서': 2,
            '임차권이전': 2, '금융기관담보': 1, '임차권등기명령': 3,
            '등기촉탁': 2, '기입등기': 2, '전자확정일자': 3,
        },

        # 2. 계약갱신_연장: 갱신요구권 및 실거주 거절 (빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1, '2+2': 2,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1, '직계존비속': 2
        },

        # 3. 수선_원상회복: 민법 제615조 기반의 유지보수 및 청소비 공제
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '원상에 회복': 3, '파손': 2,
            '훼손': 2, '누수': 3, '곰팡이': 2, '보일러': 2, '필요비': 3,
            '유익비': 3, '비용상환': 2, '보존행위': 2, '청소비': 3,
            '원상복구': 2, '통상적 마멸': 3, '자연적 마멸': 2, '공제': 3,
            '공용부분': 2, '부속시설': 2, '교체': 1,
        },

        # 4. 생활환경_특약: 주거의 질과 관련된 커뮤니티 갈등 (신설)
        '생활환경_특약': {
            '층간소음': 3, '반려동물': 3, '애완동물': 3, '흡연': 3, '담배': 2,
            '주차': 2, '결로': 2, '방음': 2, '악취': 2, '공동생활': 2,
            '금지사항': 2, '입주민갈등': 2, '벽지훼손': 2,
            '평온을 해치는 행위': 3, '관리규약': 2
        },

        # 5. 관리비_투명성: 관리비 꼼수 인상 및 내역 공개 (최신 이슈)
        '관리비_투명성': {
            '관리비': 3, '공공요금': 2, '정액관리비': 3, '세부내역': 2,
            '장기수선충당금': 3, '수선유지비': 2, '관리비인상': 2, '전기료': 1,
            '수도료': 1, '비용 청구': 1, '사용료': 2, '관리주체': 2, '내역공개': 2,
            '수선적립금': 3, '관리단': 2, '분담금': 2
        },

        # 6. 임대료_증감: 차임 증액 상한(5%) 및 월세 전환율
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1, '연체': 3, '3기': 3,
            '보증금전환': 3, '임차인동의': 2,
        },

        # 7. 권리_정보리스크: 신탁 부동산, 전세사기, 세금 체납 (정보 비대칭)
        '권리_정보리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '수탁자': 2, '근저당': 3,
            '저당권': 3, '선순위': 3, '가압류': 3, '압류': 3, '국세': 2,
            '지방세': 2, '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '확정일자현황': 3, '조세채권': 2, '법정기일': 3, '미납국세열람': 2,
            '사용승인': 2, '무허가': 2, '거짓계약': 2,
            '법정기일': 3, '미납지방세열람': 3, '사해행위취소': 2,
            '본등기': 2, '직권말소': 2, '가등기': 2,
            '정보제공요청': 3, '열람신청': 2, '이해관계인 증명': 2,
        },

        # 8. 특약_유효성: 주임법 제10조(강행규정) 및 독소조항 탐지
        '특약_유효성': {
            '특약': 3, '특약사항': 3, '강행규정': 3, '제10조': 3,
            '효력이 없다': 3, '무효': 3, '불리한 약정': 3, '일방적 포기': 2,
            '편면적 강행규정': 2, '배제': 1, '표준임대차계약서': 3
        },

        # 9. 배상_책임: 민법 제390조/제750조 기반의 손해배상 및 위약금
        '손해배상_책임': {
            '손해배상': 3, '채무불이행': 3, '불법행위': 3, '위약금': 3,
            '과실': 2, '배상액': 2, '지연이자': 2, '이행지체': 2, '위반': 2
        },

        # 10. 계약해지_명도: 계약 종료 절차 및 퇴거/이사 관련
        '계약해지_명도': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 3, '이사': 1, '명도': 3, '합의해지': 2,
            '통지': 1, '3개월': 2, '즉시해지': 3, '내용증명': 2,
            '임대차신고': 3, '거래신고': 3, '신고필증': 2, '등기사항': 2, '정정신청': 2,
        },

        # 11. 행정_절차: 확정일자 부여 및 전자계약 절차
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },

        # 12. 분쟁해결_절차: 조정위원회 및 소송/경매 최후 수단
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 3, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2, '제소전화해': 3,
            '송달': 2, '강제집행': 3, '배당요구': 3, '매수신청보증': 2,
            '조정신청각하': 2, '출석요구': 1, '서면교부': 1,
            '소액사건': 3, '3,000만원': 3, '이행권고': 2, '공시최고': 1,
            '임차권등기명령신청': 3, '결정송달': 2, '신청서기재사항': 1,
        },

        # 13. 임차권_승계: 제9조 타겟팅 및 사실혼 가족 보호
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2,
            '공동상속': 2, '반환청구권': 2
        }
    }

def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    
    category_keywords = get_law_category()
    category_scores = {}
    
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0:
            category_scores[category] = score
            
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    all_categories = [category[0] for category in sorted_categories]
    
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
        
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories

In [4]:
# =====================
# 참조조항을 추출하는 함수
# =====================

_ARTICLE_RE = re.compile(r"(?:제\s*)?(\d+)\s*조(?:\s*의\s*(\d+))?")

def _law_name_from_src_title(src_title: str) -> str:
    if not src_title:
        return "알수없음"
    s = str(src_title).strip()
    if not s:
        return "알수없음"
    # 첫 '(' 이전을 법령명으로 사용
    name = s.split("(", 1)[0].strip()
    return name if name else "알수없음"

def _split_article_field(article_field) -> list[str]:
    if not article_field or article_field == "조항없음":
        return []
    s = str(article_field)
    s = s.replace("「", "").replace("」", "").strip()
    if not s:
        return []
    # ",", ";", 줄바꿈, "·" 등 다양한 구분자 대응
    tokens = re.split(r"[,\n;·]+", s)
    return [t.strip() for t in tokens if t and t.strip()]

def _parse_article_token(token: str):
    if not token:
        return None
    token = re.sub(r"\s+", "", token)
    m = _ARTICLE_RE.search(token)
    if not m:
        return None
    main = int(m.group(1))
    sub = int(m.group(2)) if m.group(2) else 0
    return (main, sub)

def _format_article_key(key):
    main, sub = key
    return f"제{main}조" if sub == 0 else f"제{main}조의{sub}"

def extract_articles_from_docs(documents):
    """
    검색된 문서들(documents)에서 article 정보를 추출하여
    '법령명: 제x조,제y조 | 법령명2: ...' 형태로 반환
    """
    # law_name -> set( (main,sub) )
    buckets = defaultdict(set)
    # law_name -> best priority(작을수록 우선)
    law_priority = {}

    for doc in documents:
        meta = doc.metadata or {}

        law = _law_name_from_src_title(meta.get("src_title") or meta.get("law_name") or meta.get("source"))
        pr = meta.get("priority", None)
        try:
            pr_int = int(pr) if pr is not None else None
        except Exception:
            pr_int = None

        if pr_int is not None:
            prev = law_priority.get(law)
            if prev is None or pr_int < prev:
                law_priority[law] = pr_int

        for tok in _split_article_field(meta.get("article", "조항없음")):
            key = _parse_article_token(tok)
            if key is not None:
                buckets[law].add(key)

    if not buckets:
        return "(참조 조항 없음)"

    # 법령 블록 정렬: priority(있으면) -> 법령명
    def law_sort_key(law_name: str):
        p = law_priority.get(law_name)
        return (p if p is not None else 9999, law_name)

    parts = []
    for law in sorted(buckets.keys(), key=law_sort_key):
        keys = sorted(buckets[law])
        if not keys:
            continue
        parts.append(f"{law} " + ",".join(_format_article_key(k) for k in keys))

    return " | ".join(parts) if parts else "(참조 조항 없음)"



# ====================================
# 검색된 document를 텍스트로 변환하는 함수
# ====================================

def format_documents(documents):
    return '\n\n---\n\n'.join([doc.page_content for doc in documents])


# ===========================
# title+text 기반의 rerank함수
# ===========================

import cohere
import numpy as np

def rerank_by_title(query: str, documents: list, top_k: int = 4):
    '''
    cohere의 rerank API를 사용한 문서 재정렬
    '''
    # cohere client 초기화
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    # 0건이면 rerank 호출하지 말고 그대로 반환
    if not documents:
        return []

    # top_k도 0이면 그대로
    if not top_k or top_k <= 0:
        return documents

    docs_text = [
        (doc.metadata.get("src_title", "") or "") + " " + (doc.page_content or "")
        for doc in documents
    ]

    # documents가 비어있지 않아도, docs_text가 전부 빈문자면 Cohere가 싫어할 수 있어서 방어
    if not any(t.strip() for t in docs_text):
        return documents[:top_k]

    results = co.rerank(
        model="rerank-multilingual-v3.0",
        query=query,
        documents=docs_text,
        top_n=min(top_k, len(docs_text)),
    )

    idxs = [r.index for r in results.results]
    return [documents[i] for i in idxs]


# ==========================================
# 검색된 document를 LLM용 context로 변환 (개선)
# ==========================================

def format_documents_with_meta(documents, max_len: int = 500):
    """
    프롬프트.txt 구조 반영:
    - 법령명 / 조문 / priority 명시
    - LLM이 법적 근거를 명확히 인식하도록 설계
    """
    context_parts = []

    for i, doc in enumerate(documents, 1):
        meta = doc.metadata or {}

        law_name = meta.get("law_name") or meta.get("src_title") or meta.get("source", "알 수 없는 법령")
        article = meta.get("article", "")
        priority = meta.get("priority", "?")

        content = doc.page_content.strip()
        if len(content) > max_len:
            content = content[:max_len].rstrip() + "…"

        context_parts.append(
            f"""[문서 {i}]
- 법령명: {law_name}
- 조문: {article}
- 우선순위(priority): {priority}
- 내용:
{content}
"""
        )

    return "\n\n".join(context_parts)

##  3중 인덱스 통합검색

- **2-stage 판례 전략**: case-index는 먼저 청크 후보를 넉넉히 가져온 뒤, 상위 사건번호(top N)만 전문(context)으로 확장
- **법적 위계 컨텍스트 구성**: priority 기준으로 SECTION 1(법령) / SECTION 2(규정·절차) / SECTION 3(판례)로 재구성

In [5]:
from dataclasses import dataclass
from typing import Iterable, Sequence, Tuple, List, Optional

def _safe_int(x: object, default: int = 99) -> int:
    try:
        return int(x)  # type: ignore[arg-type]
    except Exception:
        return default

def _truncate(text: str, max_chars: int) -> str:
    if text is None:
        return ""
    if len(text) <= max_chars:
        return text
    return text[: max_chars - 1] + "…"

def _dedupe_docs(
    docs: Iterable[Document],
    key_fields: Sequence[str] = ("chunk_id", "id"),
) -> List[Document]:
    """
    메타데이터 기반 중복 제거 (chunk_id 우선)
    - key_fields 중 첫 번째로 발견되는 값을 키로 사용
    - 키가 없으면 page_content 해시로 폴백
    """
    seen = set()
    out: List[Document] = []
    for d in docs:
        md = d.metadata or {}
        key = None
        for f in key_fields:
            v = md.get(f)
            if v:
                key = f"{f}:{v}"
                break
        if key is None:
            key = f"content:{hash(d.page_content)}"
        if key in seen:
            continue
        seen.add(key)
        out.append(d)
    return out

def _attach_source(docs: List[Document], source: str) -> List[Document]:
    """검색 출처(law/rule/case)를 메타데이터에 주입"""
    for d in docs:
        if d.metadata is None:
            d.metadata = {}
        d.metadata["__source_index"] = source
    return docs

def get_full_case_context(
    case_db: PineconeVectorStore,
    *,
    case_no: str,
    k: int = 50,
    dedupe_key_fields: Tuple[str, ...] = ("chunk_id", "id"),
) -> str:
    """특정 사건번호(case_no)의 판례 전문(청크들을 연결)을 가져옴"""
    try:
        results = case_db.similarity_search(
            query="판례 전문 검색",  # API 요구사항용 더미 쿼리
            k=k,
            filter={"case_no": {"$eq": case_no}},
        )
        # chunk_id 순 정렬 후 중복 제거
        sorted_docs = sorted(results, key=lambda x: str((x.metadata or {}).get("chunk_id", "")))
        unique_docs = _dedupe_docs(sorted_docs, dedupe_key_fields)
        return "\n".join([d.page_content for d in unique_docs]).strip()
    except Exception as e:
        print(f"⚠️ 판례 전문 로딩 실패 ({case_no}): {e}")
        return ""

@dataclass
class MultiIndexRetrievalConfig:
    # 최종 target
    k_law: int = 5
    k_rule: int = 5
    k_case: int = 3

    # oversampling
    search_multiplier: int = 2

    # 2-stage case
    case_candidate_k: int = 40
    case_expand_top_n: Optional[int] = None
    case_context_top_k: int = 50

    # dedupe
    dedupe_key_fields: Tuple[str, ...] = ("chunk_id", "id")

def format_context_with_hierarchy(docs: List[Document]) -> str:
    """priority 기반으로 섹션을 재구성"""
    section_1_law: List[str] = []   # Priority 1, 2, 4, 5
    section_2_rule: List[str] = []  # Priority 3, 6, 7, 8, 11
    section_3_case: List[str] = []  # 기타 (주로 판례)

    for doc in docs:
        md = doc.metadata or {}
        p = _safe_int(md.get("priority", 99), 99)
        src = md.get("src_title", md.get("__source_index", "자료"))
        title = md.get("title", "")
        content = doc.page_content or ""

        entry = f"[{src}] {title}\n{content}".strip()

        if p in (1, 2, 4, 5):
            section_1_law.append(entry)
        elif p in (3, 6, 7, 8, 11):
            section_2_rule.append(entry)
        else:
            section_3_case.append(entry)

    parts: List[str] = []
    if section_1_law:
        parts.append("## [SECTION 1: 핵심 법령 (최우선 법적 근거)]\n" + "\n\n".join(section_1_law))
    if section_2_rule:
        parts.append("## [SECTION 2: 관련 규정 및 절차 (세부 기준)]\n" + "\n\n".join(section_2_rule))
    if section_3_case:
        parts.append("## [SECTION 3: 판례 및 해석 사례 (적용 예시)]\n" + "\n\n".join(section_3_case))

    return "\n\n".join(parts).strip()

def triple_hybrid_retrieval_2stage_case(
    *,
    law_db: PineconeVectorStore,
    rule_db: PineconeVectorStore,
    case_db: PineconeVectorStore,
    query: str,
    cfg: Optional[MultiIndexRetrievalConfig] = None,
    # rerank 함수는 노트북에 이미 있는 rerank_by_title를 기본 사용
    rerank_fn=rerank_by_title,
    rerank_top_k: Optional[int] = None,
) -> List[Document]:
    """
    Law/Rule/Case 3중 인덱스 검색 + (선택) rerank + 2-stage 판례 확장.
    - rerank_top_k: None이면 (k_law+k_rule+case_candidate_top) 정도로 자동 설정
    """
    cfg = cfg or MultiIndexRetrievalConfig()
    mult = max(1, int(cfg.search_multiplier))

    # 1) Retrieve (oversampling)
    docs_law = _attach_source(law_db.similarity_search(query, k=cfg.k_law * mult), "law")
    docs_rule = _attach_source(rule_db.similarity_search(query, k=cfg.k_rule * mult), "rule")
    docs_case_chunks = _attach_source(case_db.similarity_search(query, k=cfg.case_candidate_k), "case")

    # 2) Combine + (optional) rerank
    combined = _dedupe_docs(docs_law + docs_rule + docs_case_chunks, cfg.dedupe_key_fields)

    if rerank_fn is not None:
        # 기존 노트북 rerank_by_title은 list[Document] -> list[Document] 반환
        _top_k = rerank_top_k if rerank_top_k is not None else min(len(combined), (cfg.k_law + cfg.k_rule + cfg.k_case) * 4)
        try:
            combined = rerank_fn(query=query, documents=combined, top_k=_top_k)
        except Exception as e:
            print(f"⚠️ rerank 스킵: {e}")

    combined = _dedupe_docs(combined, cfg.dedupe_key_fields)

    # 3) source별 top-k 유지
    law_ranked = [d for d in combined if (d.metadata or {}).get("__source_index") == "law"]
    rule_ranked = [d for d in combined if (d.metadata or {}).get("__source_index") == "rule"]
    case_ranked_chunks = [d for d in combined if (d.metadata or {}).get("__source_index") == "case"]

    final_law = law_ranked[: cfg.k_law]
    final_rule = rule_ranked[: cfg.k_rule]

    # 4) 2-stage case: unique case_no 선정 -> 전문 확장
    top_n = cfg.case_expand_top_n if cfg.case_expand_top_n is not None else cfg.k_case
    seen_case_no = set()
    chosen_case_docs: List[Document] = []
    for d in case_ranked_chunks:
        case_no = (d.metadata or {}).get("case_no")
        if not case_no or case_no in seen_case_no:
            continue
        seen_case_no.add(case_no)
        chosen_case_docs.append(d)
        if len(chosen_case_docs) >= top_n:
            break

    expanded_cases: List[Document] = []
    for d in chosen_case_docs:
        case_no = str((d.metadata or {}).get("case_no"))
        full_text = get_full_case_context(
            case_db,
            case_no=case_no,
            k=cfg.case_context_top_k,
            dedupe_key_fields=cfg.dedupe_key_fields,
        )
        if not full_text:
            expanded_cases.append(d)
            continue

        title = (d.metadata or {}).get("title") or (d.metadata or {}).get("case_name") or case_no
        md = dict(d.metadata or {})
        md["__expanded"] = True
        expanded_cases.append(
            Document(
                page_content=f"[판례 전문: {title}]\n{full_text}",
                metadata=md,
            )
        )

    final_case = expanded_cases[: cfg.k_case]

    # 5) Priority sort (법적 위계)
    final_docs = final_law + final_rule + final_case
    final_docs = sorted(final_docs, key=lambda x: _safe_int((x.metadata or {}).get("priority", 99), 99))
    return final_docs


## 우선순위

In [6]:
def sanitize_for_upstage(text: str, max_len: int = 500) -> str:
    """Upstage embedding input 안전 정제: str 보장, 공백/개행 정리, 길이 제한"""
    if text is None:
        return ""
    if not isinstance(text, str):
        text = str(text)

    text = text.strip()

    # 이상한 프롬프트 잔재 제거(가끔 '질문 :' 같은 prefix가 섞임)
    text = re.sub(r"^\s*(질문\s*:|question\s*:|q\s*:)\s*", "", text, flags=re.IGNORECASE)

    # 개행/탭 과다를 단일 공백으로
    text = re.sub(r"\s+", " ", text).strip()

    # 너무 길면 자르기
    if len(text) > max_len:
        text = text[:max_len].strip()

    return text

def safe_embedding_query(normalized_query: str, original_query: str) -> str:
    q = sanitize_for_upstage(normalized_query)
    if not q:  # 빈 문자열이면 원문으로 폴백
        q = sanitize_for_upstage(original_query)
    return q


def retrieve_law_docs_priority_mix(
    law_db,
    embedding,
    query_for_embed: str,
    categories: Optional[List[str]] = None,
    k_total: int = 6,
    k_each: int = 8,
    # ✅ 우선순위 정책: 1(법) 먼저, 2(시행령) 최소 확보, 그 다음 4(민법) 등
    priority_order: List[int] = [1, 2, 4, 3, 5, 6, 7],
    # ✅ 최소 할당량(시행령 1개 확보)
    min_quota: Optional[Dict[int, int]] = None,
):
    if min_quota is None:
        min_quota = {1: max(1, k_total - 1), 2: 1}  # 기본: 1에서 k_total-1, 2에서 1

    qvec = embedding.embed_query(query_for_embed)
    picked: List[Document] = []
    seen = set()

    def add_docs(docs: List[Document]):
        nonlocal picked
        for d in docs:
            key = (
                d.metadata.get("id"),
                d.metadata.get("chunk_id"),
                d.metadata.get("source"),
                d.page_content[:120],
            )
            if key in seen:
                continue
            seen.add(key)
            picked.append(d)

    def query_priority(p: int, k_take: int):
        # filter 구성 (category는 있으면 넣고, 없으면 생략)
        flt = {"priority": {"$in": [p, float(p)]}}  # 2 vs 2.0 방어
        if categories:
            flt["category"] = {"$in": categories}

        return law_db.similarity_search_by_vector(
            qvec,
            k=min(k_each, k_take) if k_take > 0 else k_each,
            filter=flt,
        )

    # 1) 최소 할당(min_quota) 먼저 채우기
    for p in priority_order:
        need = min_quota.get(p, 0)
        if need <= 0:
            continue
        docs = query_priority(p, need)
        add_docs(docs[:need])  # 필요한 만큼만
        if len(picked) >= k_total:
            return picked[:k_total]

    # 2) 남은 수를 priority_order 순서대로 채우기
    for p in priority_order:
        if len(picked) >= k_total:
            break
        remaining = k_total - len(picked)
        docs = query_priority(p, remaining)
        add_docs(docs)

    return picked[:k_total]


## 비용계산

In [7]:
def _normalize_token_usage(d: Dict[str, Any]) -> Optional[Dict[str, Optional[int]]]:
    """
    여러 provider 형태의 usage dict를 LangChain 표준 키로 정규화.
    표준: input_tokens / output_tokens / total_tokens
    (OpenAI 계열): prompt_tokens / completion_tokens / total_tokens
    """
    if not isinstance(d, dict) or not d:
        return None

    # 1) 이미 표준 키인 경우
    input_t = d.get("input_tokens")
    output_t = d.get("output_tokens")
    total_t = d.get("total_tokens")

    # 2) OpenAI 스타일 키인 경우
    if input_t is None:
        input_t = d.get("prompt_tokens")
    if output_t is None:
        output_t = d.get("completion_tokens")

    # 3) 다른 변형 키들(혹시 모를 케이스)
    if input_t is None:
        input_t = d.get("prompt") or d.get("input")
    if output_t is None:
        output_t = d.get("completion") or d.get("output")

    # total 없으면 계산
    if total_t is None and isinstance(input_t, int) and isinstance(output_t, int):
        total_t = input_t + output_t

    # 최소한 input/output 둘 중 하나라도 있어야 유효
    if input_t is None and output_t is None and total_t is None:
        return None

    return {
        "input_tokens": input_t if isinstance(input_t, int) else None,
        "output_tokens": output_t if isinstance(output_t, int) else None,
        "total_tokens": total_t if isinstance(total_t, int) else None,
    }


def _usage_metadata_to_dict(usage_meta: Any) -> Optional[Dict[str, Any]]:
    """
    usage_metadata가 dict일 수도 있고, UsageMetadata 객체(속성)일 수도 있어 방어적으로 dict로 변환.
    """
    if usage_meta is None:
        return None

    if isinstance(usage_meta, dict):
        return usage_meta

    # pydantic/dataclass/객체 형태 대비: 속성 접근
    out = {}
    for k in ("input_tokens", "output_tokens", "total_tokens"):
        if hasattr(usage_meta, k):
            out[k] = getattr(usage_meta, k)
    return out or None


def get_token_usage(llm: Any, llm_response: Any) -> Optional[Dict[str, Optional[int]]]:
    """
    Provider-agnostic 토큰 사용량 추출 (OpenAI/Upstage 등 공통).
    - 1순위: llm_response.usage_metadata  (LangChain 표준)
    - 2순위: llm_response.response_metadata["token_usage"] (OpenAI 계열 자주 존재)
    - 3순위: llm_response.response_metadata["usage"] / ["usage_metadata"] 등 변형
    - 그 외: None
    """
    # 1) LangChain 표준 usage_metadata
    usage_meta = _usage_metadata_to_dict(getattr(llm_response, "usage_metadata", None))
    normalized = _normalize_token_usage(usage_meta or {})
    if normalized:
        return normalized

    # 2) response_metadata 쪽에서 찾기
    meta = getattr(llm_response, "response_metadata", None) or {}
    if isinstance(meta, dict):
        # 가장 흔한 케이스: token_usage
        normalized = _normalize_token_usage(meta.get("token_usage") or {})
        if normalized:
            return normalized

        # provider에 따라 usage 라는 이름을 쓰는 케이스들
        normalized = _normalize_token_usage(meta.get("usage") or {})
        if normalized:
            return normalized

        normalized = _normalize_token_usage(meta.get("usage_metadata") or {})
        if normalized:
            return normalized

        # 깊게 한번 더(혹시 중첩된 경우)
        for key in ("response", "raw", "data", "result"):
            nested = meta.get(key)
            if isinstance(nested, dict):
                for k2 in ("token_usage", "usage", "usage_metadata"):
                    normalized = _normalize_token_usage(nested.get(k2) or {})
                    if normalized:
                        return normalized

    return None


OPENAI_PRICES = {
    "gpt-4o-mini": {"input": 0.15 / 1000000,"output": 0.60 / 1000000},
    "gpt-4.1-mini": { "input": 0.40 / 1000000, "output": 1.60 / 1000000},
    "gpt-5-mini": {"input": 0.25 / 1000000, "output": 2.00 / 1000000},
    "gpt-5-nano": { "input": 0.05 / 1000000, "output": 0.40 / 1000000},
    "gpt-4.1-nano": { "input": 0.10 / 1000000, "output": 0.40 / 1000000},  
}

UPSTAGE_PRICES = {
    "solar-pro2": { "input": 0.15 / 1000000, "output": 0.60 / 1000000},
}

ALL_PRICES = {**OPENAI_PRICES, **UPSTAGE_PRICES}

def calculate_llm_cost(model_name: str, token_usage: dict) -> float | None:
    """
    OpenAI + Upstage 공통 비용 계산.
    """
    if not token_usage:
        return None

    price = ALL_PRICES.get(model_name)
    if not price:
        return None

    in_tok = token_usage.get("input_tokens")
    out_tok = token_usage.get("output_tokens")
    if not isinstance(in_tok, int) or not isinstance(out_tok, int):
        return None

    return in_tok * price["input"] + out_tok * price["output"]

## Query Planner

In [8]:
def extract_json_object(text: str) -> Optional[Dict[str, Any]]:
    """
    LLM 출력에서 가장 바깥 JSON object({...}) 하나를 찾아 파싱.
    실패하면 None.
    """
    if not text:
        return None

    # 가장 단순하지만 실전에 강한 방식: 첫 { 와 마지막 } 사이를 후보로
    first = text.find("{")
    last = text.rfind("}")
    if first == -1 or last == -1 or last <= first:
        return None

    candidate = text[first:last+1].strip()
    try:
        return json.loads(candidate)
    except Exception:
        # 혹시 코드블록 ```json ... ``` 형태 제거 후 재시도
        candidate2 = re.sub(r"```(?:json)?\s*|\s*```", "", candidate, flags=re.IGNORECASE).strip()
        try:
            return json.loads(candidate2)
        except Exception:
            return None

        
def validate_and_normalize_plan(plan: Dict[str, Any]) -> Dict[str, Any]:
    """
    최소한의 규칙으로 plan을 정리하고, 사용할지 말지 결정하는 플래그 포함해서 반환.
    반환 예:
      {
        "use_plan": True/False,
        "confidence": 0.0~1.0,
        "queries": [...],
        "issues": [...],
        "law_types": [...],
        "need_facts": [...],
        "reason": "..."
      }
    """
    out = {
        "use_plan": False,
        "confidence": 0.0,
        "queries": [],
        "issues": [],
        "law_types": [],
        "need_facts": [],
        "reason": ""
    }
    if not isinstance(plan, dict):
        out["reason"] = "plan_not_dict"
        return out

    # 필드 읽기
    confidence = plan.get("confidence", 0)
    queries = plan.get("queries", [])
    issues = plan.get("issues", [])
    law_types = plan.get("law_types", [])
    need_facts = plan.get("need_facts", [])

    # 타입/값 정리
    try:
        confidence = float(confidence)
    except Exception:
        confidence = 0.0
    confidence = max(0.0, min(1.0, confidence))

    if not isinstance(queries, list): queries = []
    if not isinstance(issues, list): issues = []
    if not isinstance(law_types, list): law_types = []
    if not isinstance(need_facts, list): need_facts = []

    # 문자열만 남기고 정리
    queries = [str(x).strip() for x in queries if str(x).strip()]
    issues = [str(x).strip() for x in issues if str(x).strip()]
    law_types = [str(x).strip() for x in law_types if str(x).strip()]
    need_facts = [str(x).strip() for x in need_facts if str(x).strip()]

    # 길이 제한(과도한 확장 방지)
    queries = queries[:5]
    issues = issues[:4]
    law_types = law_types[:4]
    need_facts = need_facts[:6]

    # law_types 화이트리스트(원하는 것만)
    allowed_law_types = {
        "주택임대차보호법", "주택임대차보호법 시행령", "민법", "판례", "기타"
    }
    law_types = [x for x in law_types if x in allowed_law_types]

    # 쿼리 최소 품질: 너무 짧으면 버림
    queries = [q for q in queries if len(q) >= 2]

    # 사용 여부 판단
    # - confidence가 낮거나
    # - queries가 비어있으면 폴백
    if confidence >= 0.55 and len(queries) >= 1:
        out["use_plan"] = True
    else:
        out["use_plan"] = False

    out["confidence"] = confidence
    out["queries"] = queries
    out["issues"] = issues
    out["law_types"] = law_types
    out["need_facts"] = need_facts
    out["reason"] = plan.get("reason", "")

    return out

## RAGAS

In [9]:
import re
from dataclasses import dataclass
from typing import List, Tuple, Set, Dict, Optional

# 답변에서 조문(조/조의) 탐지
_ART_IN_ANSWER_RE = re.compile(r"(제\s*\d+\s*조(?:\s*의\s*\d+)?)")

# 문장 분리(간단): 줄바꿈/마침표/물음표/느낌표/‘다.’ 기준
_SENT_SPLIT_RE = re.compile(r"(?:(?<=[\.\!\?])\s+|(?<=다\.)\s+|\n+)")

def _norm_art(s: str) -> str:
    # "제 6 조 의 2" -> "제6조의2"
    return re.sub(r"\s+", "", s).replace("조의", "조의")

def _norm_article_token(tok: str) -> Optional[str]:
    """
    메타데이터 article 토큰을 '제N조' or '제N조의M'으로 정규화
    입력 예: "20조", "21 조", "제22조", "제 6 조 의 2"
    """
    if not tok:
        return None
    t = re.sub(r"\s+", "", str(tok))
    m = re.search(r"(?:제)?(\d+)조(?:의(\d+))?", t)
    if not m:
        return None
    main = m.group(1)
    sub = m.group(2)
    return f"제{main}조" if not sub else f"제{main}조의{sub}"

def _law_name_from_doc_meta(meta: dict) -> str:
    # 네 코드에 이미 src_title에서 법령명 추출 로직이 있음(첫 '(' 이전). :contentReference[oaicite:2]{index=2}
    src_title = (meta or {}).get("src_title") or ""
    return _law_name_from_src_title(src_title)

def _split_article_field_local(article_field) -> List[str]:
    # 네 코드에 이미 구분자 split 로직이 있음. :contentReference[oaicite:3]{index=3}
    return _split_article_field(article_field)

def build_allowed_citations_from_docs(docs) -> Set[Tuple[str, str]]:
    """
    retrieved_docs metadata 기반으로 허용 인용 집합 생성:
      (법령명, '제N조/제N조의M')
    """
    allowed: Set[Tuple[str, str]] = set()
    for d in docs:
        meta = getattr(d, "metadata", {}) or {}
        law = _law_name_from_doc_meta(meta)
        for tok in _split_article_field_local(meta.get("article", None)):
            art = _norm_article_token(tok)
            if art:
                allowed.add((law, art))
    return allowed

def extract_answer_citations(answer: str, law_names: List[str]) -> List[Tuple[str, str]]:
    """
    답변에서 (법령명, 조문) 추출.
    law_names는 docs에서 나온 법령명 목록을 넣어 정확도를 높임.
    """
    if not law_names:
        return []
    # 길이 긴 법령명 먼저
    law_names = sorted(set(law_names), key=len, reverse=True)
    law_part = "(" + "|".join(map(re.escape, law_names)) + ")"
    re_law_art = re.compile(rf"(?P<law>{law_part})\s*(?P<art>제\s*\d+\s*조(?:\s*의\s*\d+)?)")
    out = []
    for m in re_law_art.finditer(answer or ""):
        out.append((m.group("law").strip(), _norm_art(m.group("art"))))
    return out

def extract_art_mentions(answer: str) -> List[str]:
    return [_norm_art(m.group(1)) for m in _ART_IN_ANSWER_RE.finditer(answer or "")]

@dataclass
class RuleGateResult:
    completeness: float
    accuracy: float
    n_answer_citations: int
    art_only_sentences: List[str]              # 법령명 없이 조문만 나온 문장들(디버그)
    inaccurate_citations: List[Tuple[str,str]] # (law, art) not in allowed
    debug: Dict

def rule_gate_eval(answer: str, docs) -> RuleGateResult:
    # 1) docs에서 법령명 목록 수집
    law_names = []
    for d in docs:
        meta = getattr(d, "metadata", {}) or {}
        law_names.append(_law_name_from_doc_meta(meta))
    law_names = sorted(set([x for x in law_names if x and x != "알수없음"]), key=len, reverse=True)

    # 2) 허용 인용 집합(메타 기반)
    allowed = build_allowed_citations_from_docs(docs)

    # 3) 답변 인용 추출
    answer_cits = extract_answer_citations(answer, law_names)

    # 4) Accuracy
    if not answer_cits:
        accuracy = 1.0
        inaccurate = []
    else:
        inaccurate = [(law, art) for (law, art) in answer_cits if (law, art) not in allowed]
        accuracy = (len(answer_cits) - len(inaccurate)) / len(answer_cits)

    # 5) Completeness (문장 단위로 엄격하게)
    sentences = [s.strip() for s in _SENT_SPLIT_RE.split(answer or "") if s and s.strip()]
    art_only_sents = []
    total_art_mentions = 0
    ok_art_mentions = 0

    for s in sentences:
        arts = extract_art_mentions(s)
        if not arts:
            continue
        total_art_mentions += len(arts)
        has_law = any(law in s for law in law_names)
        # 문장 안에 법령명이 있으면, 그 문장의 조문 언급은 모두 "완전" 처리
        if has_law:
            ok_art_mentions += len(arts)
        else:
            # 법령명 없는 문장인데 조문 언급이 있음 → completeness 실패 원인
            art_only_sents.append(s)

    completeness = 1.0 if total_art_mentions == 0 else (ok_art_mentions / total_art_mentions)

    return RuleGateResult(
        completeness=completeness,
        accuracy=accuracy,
        n_answer_citations=len(answer_cits),
        art_only_sentences=art_only_sents,
        inaccurate_citations=inaccurate,
        debug={
            "law_names": law_names[:30],
            "allowed_sample": list(sorted(allowed))[:30],
            "answer_citations": answer_cits,
            "total_art_mentions": total_art_mentions,
        }
    )

def pass_rule_gate(rule: RuleGateResult, completeness_min=0.99, accuracy_min=0.90) -> bool:
    if rule.completeness < completeness_min:
        return False
    if rule.accuracy < accuracy_min:
        return False
    return True

# 2. LLM

In [12]:
def ask_with_evaluation(query: str, k: int = 4, top_k: int = 4):
    """query, 표준화된 query, 답변, 참조사항 출력"""

    # ==================
    # 1. LLM / Embedding
    # ==================
    
    # - Planner는 가벼운 로컬(ollama), 최종 답변은 upstage 유지
#     planner_llm = ChatOllama(model="exaone3.5:2.4b", temperature=0)
    planner_llm = ChatUpstage(model="solar-pro2")
    response_llm = ChatUpstage(model="solar-pro2")
    up_embedding = UpstageEmbeddings(model=UPSTAGE_EMBEDDIGN_MODEL)

    
    # =============
    # 2. Vector DB 
    # =============
    
    # Priority 1,2,4,5: 주임법, 민법 등 핵심 법률
    law_database = PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_LAW)
    
    # Priority 3,6,7,8,11: 시행규칙, 조례, 절차
    rule_database = PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_RULE)

    # Priority 9: 판례, 상담사례
    case_database = PineconeVectorStore(
        embedding=up_embedding,
        index_name=PINECONE_INDEX_NAME_CASE)
    
    
    # ===============
    # 3. Keyword dict
    # ===============
    
    KEYWORD_DICT = {
        # 1. 계약 주체 및 대상
        "집주인": "임대인", "건물주": "임대인", "주인집": "임대인", "임대업자": "임대인", "새주인": "임대인",
        "세입자": "임차인", "월세입자": "임차인", "세들어사는사람": "임차인", "임차자": "임차인", "입주자": "임차인",
        "부동산": "공인중개사", "중개인": "공인중개사", "중개소": "공인중개사",
        "빌라": "임차주택", "아파트": "임차주택", "오피스텔": "임차주택", "우리집": "임차주택", "거주지": "임차주택",
        "계약서": "임대차계약증서", "집문서": "임대차계약증서", "종이": "임대차계약증서",

        # 2. 보증금 및 금전
        "전세금": "임차보증금", "보증금": "임차보증금", "맡긴돈": "임차보증금", "떼인돈": "임차보증금",
        "월세": "차임", "방세": "차임", "다달이내는지출": "차임", "렌트비": "차임", "임대료": "차임",
        "복비": "중개보수", "수수료": "중개보수", "중개비": "중개보수",
        "월세올리기": "차임증액", "인상": "증액", "더달라고함": "증액", "5프로": "5퍼센트상한",
        "월세깎기": "차임감액", "할인": "감액", "내리기": "감액",
        "돈먼저받기": "우선변제권", "순위": "우선변제권", "안전장치": "대항력", "돌려받기": "보증금반환",
        "보험": "반환보증", "허그": "HUG", "나라보증": "보증보험",

        # 3. 계약 상태 및 변화
        "연장하기": "계약갱신요구권", "한번더살기": "계약갱신", "2플러스2": "계약갱신요구권", "갱신": "계약갱신",
        "재계약": "계약갱신", "자동연장": "묵시적갱신", "연락없음": "묵시적갱신", "그냥연장": "묵시적갱신",
        "이사": "주택의인도", "짐빼기": "주택의인도", "퇴거": "주택의인도", "방빼": "계약해지",
        "주소옮기기": "주민등록", "전입신고": "주민등록", "주소지이전": "주민등록",
        "집주인바뀜": "임대인지위승계", "주인바뀜": "임대인지위승계", "매매": "임대인지위승계",
        "나가라고함": "계약갱신거절", "쫓겨남": "명도", "비워달라": "명도", "중도해지": "계약해지",

        # 4. 수리 및 생활환경
        "집고치기": "수선의무", "수리": "수선의무", "고쳐줘": "수선의무", "안고쳐줌": "수선의무위반",
        "곰팡이": "하자", "물샘": "누수", "보일러고장": "하자", "파손": "훼손",
        "깨끗이치우기": "원상회복의무", "원래대로해놓기": "원상회복", "청소비": "원상회복비용", "청소": "원상회복",
        "층간소음": "공동생활평온", "옆집소음": "방음", "개키우기": "반려동물특약", "담배": "흡연금지특약",

        # 5. 리스크 및 분쟁
        "깡통전세": "전세피해", "사기": "전세사기", "경매넘어감": "권리리스크", "빚": "근저당",
        "세금안냄": "체납", "나라빚": "조세채권", "빌린돈": "가압류", "신탁": "신탁부동산",
        "특약": "특약사항", "불공정": "강행규정위반", "독소조항": "불리한약정", "효력있나": "무효여부",
        "조정위": "주택임대차분쟁조정위원회", "소송말고": "분쟁조정", "법원가기싫음": "분쟁조정",
        "집주인사망": "임차권승계", "자식상속": "임차권승계"
    }

    
    # ======================================
    # 4. deterministic 표준화 (LLM 전처리 제거)
    # ======================================
    
    def normalize_query_deterministic(text: str, mapping: dict) -> str:
        # 긴 키부터 치환(부분 문자열 충돌 방지)
        keys = sorted(mapping.keys(), key=len, reverse=True)
        pattern = re.compile("|".join(map(re.escape, keys)))

        def repl(m):
            return mapping.get(m.group(0), m.group(0))

        return pattern.sub(repl, text)

    normalized_query = normalize_query_deterministic(query, KEYWORD_DICT)
    embed_query = safe_embedding_query(normalized_query, query)

    
    # ====================================================
    # 5. Planner (그대로 유지하되, 쿼리 개수 제한으로 속도 개선)
    # ====================================================
    
    planner_template = """
    너는 '주택 임대차(전월세) 법률 RAG 챗봇'의 Query Planner다.
    목표: 사용자의 질문을 보고, 벡터DB에서 관련 법령/판례 청킹을 잘 찾기 위한 "검색 계획"만 만든다.

    반드시 아래 JSON 스키마로만 출력해라. 다른 문장/설명/마크다운 금지.

    JSON 스키마:
    {{
      "confidence": 0.0,
      "issues": ["최대4개, 핵심 쟁점 키워드"],
      "law_types": ["주택임대차보호법"],
      "queries": ["최대5개, 벡터검색에 넣을 한국어 검색문/키워드 조합"],
      "need_facts": ["최대6개, 답변을 위해 추가로 필요할 수 있는 사실관계 질문 항목"],
      "reason": "짧게(20자 이내)"
    }}

    규칙:
    - queries는 서로 다른 관점으로 작성
    - 조문 번호를 억지로 만들지 마라(모르면 쓰지 않음)
    - 질문이 모호하면 confidence를 낮추고 need_facts를 채워라
    - 출력은 JSON만

    사용자 질문:
    {question}
    """
    planner_prompt = ChatPromptTemplate.from_template(planner_template)
    planner_chain = planner_prompt | planner_llm | StrOutputParser()

    plan_raw = planner_chain.invoke({"question": normalized_query})
    plan_dict = extract_json_object(plan_raw)
    plan = validate_and_normalize_plan(plan_dict)

    # 디버깅 출력(원치 않으면 주석)
    print("use_plan:", plan["use_plan"], "confidence:", plan["confidence"])
    print("queries:", plan["queries"])
    print("need_facts:", plan["need_facts"])

    # Planner 쿼리 개수 제한(속도 개선)
    max_plan_queries = 3
    if plan.get("queries"):
        plan["queries"] = plan["queries"][:max_plan_queries]

        
    # ============================
    # 6. Retrieval: 멀티쿼리 + 폴백
    # ============================
    all_docs = []

    if plan["use_plan"]:
        for q in plan["queries"]:
            # 과한 전처리/추가 호출 줄이기: q 자체를 embed에 사용
            q_for_embed = safe_embedding_query(q, normalized_query)

            docs = retrieve_law_docs_priority_mix(
                law_db=law_database,
                embedding=up_embedding,
                query_for_embed=q_for_embed,
                categories=None,
                k_total=max(k, 6),
                k_each=8,
                priority_order=[1, 2, 3, 4, 5, 6, 7],
                min_quota={1: max(1, k - 1), 2: 1},
            )
            if docs:
                all_docs.extend(docs)

        # 중복 제거
        uniq = {}
        for d in all_docs:
            key = (getattr(d, "page_content", "")[:200], str(getattr(d, "metadata", ""))[:200])
            uniq[key] = d
        retrieved_docs = list(uniq.values())

        # 부족하면 폴백
        if len(retrieved_docs) < max(3, k):
            retrieved_docs = retrieve_law_docs_priority_mix(
                law_db=law_database,
                embedding=up_embedding,
                query_for_embed=embed_query,
                categories=None,
                k_total=max(k, 6),
                k_each=8,
                priority_order=[1, 2, 3, 4, 5, 6, 7],
                min_quota={1: max(1, k - 1), 2: 1},
            )
    else:
        retrieved_docs = retrieve_law_docs_priority_mix(
            law_db=law_database,
            embedding=up_embedding,
            query_for_embed=embed_query,
            categories=None,
            k_total=max(k, 6),
            k_each=8,
            priority_order=[1, 2, 3, 4, 5, 6, 7],
            min_quota={1: max(1, k - 1), 2: 1},
        )

        
    # ==========
    # 7. Rerank
    # ==========
    
    reranked_docs = rerank_by_title(embed_query, retrieved_docs, top_k)
    referenced_articles = extract_articles_from_docs(reranked_docs)

    
    # ===============
    # 8. Context 생성
    # ===============
    
    context = format_documents_with_meta(reranked_docs)
    

    # ============
    # 9. 최종 답변
    # ============
    
    template = """
    당신은 친절하고 전문적인 주택임대차 전문 변호사입니다.
    ### 답변 작성 규칙:
    1. **질문의 핵심에 먼저 직접 답변**하세요 (예: "네, 나가셔야 합니다" 또는 "아니요, 안 나가셔도 됩니다")
    2. **이유를 쉽고 간결하게** 설명하세요
    3. **근거 법령을 자연스럽게** 언급하세요  
       [답변 생성 원칙]
        1. **법적 위계 준수**:
           - 반드시 [SECTION 1: 핵심 법령]의 내용을 최우선 판단 기준으로 삼으세요.
           - [SECTION 1]의 내용이 모호할 때만 [SECTION 2]와 [SECTION 3]를 보충 근거로 활용하세요.
           - 만약 [SECTION 3: 판례]가 [SECTION 1: 법령]과 다르게 해석되는 특수한 경우라면,
             "원칙은 법령에 따르나, 판례는 예외적으로..."라고 설명하세요.

        2. **답변 구조**:
           - **핵심 결론**: 질문에 대한 결론(가능/불가능/유효/무효)을 두괄식으로 요약.
           - **법적 근거**: "주택임대차보호법 제O조에 따르면..." (SECTION 1 인용)
           - **실무 절차**: 필요시 신고 방법, 서류 등 안내 (SECTION 2 인용)
           - **참고 사례**: 유사한 상황에서의 판결이나 해석 (SECTION 3 인용)

        3. **주의사항**:
           - 사용자의 계약서 내용이 법령(강행규정)에 위반되면 "효력이 없다(무효)"고 명확히 경고하세요.
           - 법률적 조언일 뿐이므로, 최종적으로는 변호사 등의 전문가 확인이 필요함을 고지하세요.
    4. 친근하고 이해하기 쉬운 말투를 사용하세요
    5. 임차인에게 불리한 내용은 **명확히 강조**하세요

    ### 검색된 법령 및 판례:
    {context}

    ### 질문:
    {question}

    ### 답변 (핵심 답 → 이유 → 근거 → 실행 방법 순서로):

    ### 추가로 확인하면 정확해지는 사실관계(있다면):
    {need_facts}
    """
    prompt = ChatPromptTemplate.from_template(template)
    prompt_chain = prompt | response_llm
    llm_response = prompt_chain.invoke({
        "context": context,
        "question": query,
        "need_facts": "\n- " + "\n- ".join(plan["need_facts"]) if plan.get("need_facts") else "없음"
    })
    result = llm_response.content

    
    # ==========
    # 10. RAGAS 
    # ==========

    rule = rule_gate_eval(result, reranked_docs)
    rule_pass = pass_rule_gate(rule)

    
    # =================
    # 11. 비용/요약 출력
    # =================
    
    token_usage = get_token_usage(response_llm, llm_response)
    cost = calculate_llm_cost("solar-pro2", token_usage)

    def _fmt_money(x):
        try:
            return f"{float(x):.8f}"
        except:
            return str(x)

    def _short(s, n=80):
        s = str(s)
        return (s[:n] + "…") if len(s) > n else s

    issues = plan.get("issues") or []
    need_facts = plan.get("need_facts") or []
    use_plan = bool(plan.get("use_plan"))
    conf = plan.get("confidence")
    ref_preview = _short(referenced_articles, 120)

    fail_reasons = []
    if not rule_pass:
        if rule.completeness < 0.99:
            fail_reasons.append(f"법령명 누락 문장 {len(rule.art_only_sentences)}개")
        if rule.accuracy < 0.90:
            fail_reasons.append(f"컨텍스트 없는 인용 {len(rule.inaccurate_citations)}개")
    fail_reason_text = " / ".join(fail_reasons) if fail_reasons else "-"

    print("\n" + "=" * 90)
    print("🧪 EVAL SUMMARY")
    print("-" * 90)
    print(f"Q(original)   : {_short(query, 90)}")
    print(f"Q(normalized) : {_short(normalized_query, 90)}")
    print(f"Planner       : use_plan={use_plan} | confidence={conf} | issues={', '.join(issues[:4]) if issues else '-'}")
    print(f"Need facts    : {len(need_facts)} item(s)")
    print(f"Retrieval     : k={k} | top_k={top_k} | pool={len(retrieved_docs)} | final={len(reranked_docs)}")
    print(f"Ref articles  : {ref_preview}")
    print("-" * 90)
    print(f"  - completeness: {rule.completeness:.3f} (min=0.99)")
    print(f"  - accuracy    : {rule.accuracy:.3f} (min=0.90)")
    print(f"  - fail reason : {fail_reason_text}")
    if (not rule_pass) and (rule.completeness < 0.99) and rule.art_only_sentences:
        print(f"  - example(art-only): {_short(rule.art_only_sentences[0], 120)}")
    if (not rule_pass) and (rule.accuracy < 0.90) and rule.inaccurate_citations:
        law, art = rule.inaccurate_citations[0]
        print(f"  - example(bad cite): {law} {art}")
    print("-" * 90)

    if token_usage is None:
        print("Tokens        : None")
    else:
        print(f"Tokens        : in={token_usage['input_tokens']} | out={token_usage['output_tokens']} | total={token_usage['total_tokens']}")
    print(f"Cost(est.)    : {_fmt_money(cost)}")
    print("=" * 90)
    print(result)

    return None


### 답변 비교

In [13]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제3조 ― 대항력 (전입신고·점유) ↔ 시행령 제2조·제4조·제5조·제6조)

ask_with_evaluation('전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?')

use_plan: True confidence: 0.8
queries: ['확정일자부 기재사항 확인 필요성', '확정일자와 임대차 대항력 관계', '확정일자 분쟁 시 증명력', '주택임대차보호법 확정일자 요건', '임대차 계약 확정일자 효력']
need_facts: ['임대차 계약 일자', '확정일자 부여 일자', '확정일자부 실제 기재 내용', '임대차 보증금 및 월세액', '임대인/임차인 주소 정보', '계약 갱신 여부']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 전입신고랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?
Q(normalized) : 주민등록랑 확정일자는 했는데, 확정일자부에 뭐가 어떻게 적혀 있는지까지 확인해야 하나요? 나중에 문제 생기면 그 내용도 중요해지나요?
Planner       : use_plan=True | confidence=0.8 | issues=확정일자 효력, 임대차 대항력, 확정일자부 기재사항, 분쟁 시 증명력
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=11 | final=4
Ref articles  : 주택임대차보호법 제3조,제5조,제6조,제24조,제74조,제108조,제109조,제621조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 0.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개 / 컨텍스트 없는 인용 1개
  - example(art-only): **제

In [12]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제4조 ― 임대차기간(2년 보장)

ask_with_evaluation('계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?')

use_plan: True confidence: 0.8
queries: ['임대차계약증서 기간 명시', '임대차보호법 제6조 적용 여부', '계약 조건에 따른 자동갱신 여부', '계약자 동의 없이의 강제퇴거 제한', '임대인과 임차인 간 합의 필요성']
need_facts: ['계약증서에 명시된 구체적인 기간', '임대인과 임차인 간의 합의 내용', '지역별 임대차 보호법의 세부 조항', '계약자의 현재 거주 상황', '임대인의 추가 요구 사항 여부']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 계약서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?
Q(normalized) : 임대차계약증서에 1년만 살기로 써 있는데, 1년 지나면 무조건 나가야 하는 건가요?
Planner       : use_plan=True | confidence=0.8 | issues=임대기간, 계약조건, 강제퇴거
Need facts    : 5 item(s)
Retrieval     : k=4 | top_k=4 | pool=12 | final=4
Ref articles  : 주택임대차보호법 제4조 | 주택임대차보호법 시행령 제2조,제3조,제7조,제8조,제10조,제15조,제17조
------------------------------------------------------------------------------------------
  - completeness: 0.800 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개
  - example(art-only): - **지역별 보호 한도**: 보증금 규모에 따라 우선변제권 범위가 달라질 수 있습니다(시행령 제10조).
----------------

In [13]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제5조 ― 임차인의 전대 제한

ask_with_evaluation('제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?')

use_plan: True confidence: 0.7
queries: ['친구 대신 거주 동의 여부', '임대인과의 구두 계약 유효성', '임대차 계약 조건에 친구 거주 명시 여부', '보증금 반환 및 처리 관련 조항', '임대 기간 변경 시 임대인 동의 필요성']
need_facts: ['임대인과의 구체적인 대화 내용', '현재 임대차 계약서 내용', '친구의 거주 기간 예상', '보증금 금액 및 반환 조건', '임대인의 법적 입장 표명 여부']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 집주인이랑 말로만 얘기했어요. 문제 될 수 있나요?
Q(normalized) : 제가 잠깐 해외에 나가는데, 그동안 친구가 대신 살아도 된다고 임대인이랑 말로만 얘기했어요. 문제 될 수 있나요?
Planner       : use_plan=True | confidence=0.7 | issues=임대인 동의 여부, 계약 조건, 법적 책임, 보증금 처리
Need facts    : 5 item(s)
Retrieval     : k=4 | top_k=4 | pool=12 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 부동산 거래신고 등에 관한 법률 시행령 제2조,제3조 | 민법 제618조,제628조,제638조,제640조,제646조,제648조,제650조,제653조
------------------------------------------------------------------------------------------
  - completeness: 1.000 (min=0.99)
  - accuracy    : 0.500 (min=0.90)
  - fail reason : 컨텍스트 없는 인용 1개
  - 

In [14]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제 6조(계약의 갱신)

ask_with_evaluation('집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?')

use_plan: True confidence: 0.8
queries: ['임대인 계약종료 한 달 전 임차인 변경', '임대인 임차인 변경 후 강제퇴거 가능성', '주택임대차보호법 제6조 계약종료 조건', '임대인 임차인 변경 시 임대인 의무사항', '계약종료 전 임차인 변경 시 임대인의 권리 제한']
need_facts: ['현재 계약 기간 및 만료 예정일', '임대인과 임차인 간의 계약서 내용 확인 필요', '임대인이 임차인 변경을 요구한 구체적인 이유', '지역별 주택임대차보호법 적용 여부', '이전 임차인과의 계약 내용 비교', '임대인의 강제퇴거 시도 여부 확인']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집주인이 계약 끝나기 한 달 전인데, 이미 다음 세입자를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
Q(normalized) : 임대인이 계약 끝나기 한 달 전인데, 이미 다음 임차인를 구했다면서 무조건 나가라고 합니다. 그래도 나가야 하나요?
Planner       : use_plan=True | confidence=0.8 | issues=계약종료, 임차인 변경, 강제퇴거, 계약 조건 위반
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=10 | final=4
Ref articles  : 주택임대차보호법 제6조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조 | 민법 제635조,제638조
------------------------------------------------------------------------------------------
  - completeness: 0.600 (min=0.99)
  - accuracy    : 0.667 (min=0.90)
  - fa

In [15]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제6조의3 ― 계약갱신 거절 사유(실거주 등)↔ 시행령 제5조 제5호

ask_with_evaluation('집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?')

use_plan: True confidence: 0.8
queries: ['임대인 실거주 확인 절차', '계약갱신 거절 사유 관련 판례 검색', '임대인 실거주 증명 방법', '임대인 실거주 기록 보관 방법', '임대인 실거주 증명 관련 법률 조항 검색']
need_facts: ['임대인과의 계약 내용 확인 필요', '계약 갱신 거절 사유 구체적 확인', '임대인의 실거주 여부 직접 확인 가능 여부', '증거 확보 방법에 대한 추가 정보', '관련 판례나 법률 조항의 구체적 내용 필요']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집주인이 실거주한다고 갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?
Q(normalized) : 임대인이 실거주한다고 계약갱신 거절했는데요. 나중에 실제로 안 살면, 제가 그 기록 같은 걸 확인할 수 있나요?
Planner       : use_plan=True | confidence=0.8 | issues=실거주 여부 확인, 계약갱신 거절 사유, 법적 증거 확보 필요
Need facts    : 5 item(s)
Retrieval     : k=4 | top_k=4 | pool=11 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 0.000 (min=0.90)
  - fail reason : 법령명 누락 문장 1개 / 컨텍스트 없는 인용 1개
  - example(art-only)

In [16]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조 ― 차임 증감 청구(5% 상한) ↔ 시행령 제8조

ask_with_evaluation('재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 한 번만 올릴 수 있다는 게 진짜인가요?')

use_plan: True confidence: 0.8
queries: ['계약갱신 시 차임 인상 허용 범위', '임대인 차임 인상 주기 규정', '임대차 계약 갱신 후 차임 인상 가능 기간', '임대차 보호법상 차임 인상 제한 사항', '6개월 이내 차임 인상 허용 여부 확인']
need_facts: ['현재 계약의 갱신 조건 확인', '임대인과의 계약서 상의 차임 인상 조항 확인', '지역별 주택임대차보호법 적용 여부', '최근 판례 중 차임 인상 관련 사례 분석', '임대인과의 협의 내용 기록 여부', '법률 자문을 통한 구체적인 해석 필요']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 재계약하면서 월세를 한 번에 많이 올리자고 하는데, 집주인이 정하는 대로 따라야 하나요? 그리고 월세를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에 …
Q(normalized) : 계약갱신하면서 차임를 한 번에 많이 올리자고 하는데, 임대인이 정하는 대로 따라야 하나요? 그리고 차임를 올린 지 6개월밖에 안 됐는데 또 올리자고 해요. 1년에…
Planner       : use_plan=True | confidence=0.8 | issues=계약갱신, 차임인상, 법규준수, 기간제한
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=8 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 부동산 거래신고 등에 관한 법률 시행령 제4조,제6조 | 민법 제620조
------------------------------------------------------------------------------------------
  - completeness: 0.833 (min=0.99)
  - accura

In [17]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제7조의2 ― 보증금 ↔ 월세 전환 비율 ↔ 시행령 제9조

ask_with_evaluation('보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?')

use_plan: True confidence: 0.8
queries: ['보증금 감소 후 차임 조정 허용 범위', '임대차 계약 시 차임 상한 기준', '과도한 차임 요구 시 법적 제재 사항', '임대차 계약 당사자 합의에 따른 차임 조정 절차', '임대차 보호법상 차임 조정 시 고려사항']
need_facts: ['현재 계약의 보증금 금액', '현재 차임 금액', '임대차 계약 기간', '지역별 차임 조정 기준 차이 확인 필요', '과거 판례나 관련 법률 개정 사항 확인']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 보증금 줄여주는 대신 월세로 바꾸자는데, 월세를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
Q(normalized) : 임차보증금 줄여주는 대신 차임로 바꾸자는데, 차임를 너무 많이 받으려고 해요. 기준 같은 게 있나요?
Planner       : use_plan=True | confidence=0.8 | issues=보증금 감소, 차임 조정, 과도한 차임 요구
Need facts    : 5 item(s)
Retrieval     : k=4 | top_k=4 | pool=8 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 0.000 (min=0.90)
  - fail reason : 법령명 누락 문장 2개 / 컨텍스트 없는 인용 2개
  - example(art-only): - **동법 제7조 제2항**: 증액 청구는 기존

In [18]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제8조 ― 보증금 중 일정액 보호(소액임차인) ↔ 시행령 제10조·제11조

ask_with_evaluation('집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?')

use_plan: True confidence: 0.8
queries: ['경매 진행 상황 확인', '보증금 반환 가능성 관련 판례 검색', '지역별 임차보증금 반환 기준 비교', '경매 시 보증금 우선 지급 여부 법령 검색', '보증금 반환 금액 상한 기준 검색']
need_facts: ['현재 경매 진행 단계 확인 필요', '지역별 법률 해석 차이 확인', '구체적인 임차 계약 조건 확인', '경매 절차에 따른 보증금 처리 규정 확인', '법률 자문을 통한 구체적 금액 범위 확인', '계약 당사자 간 합의 사항 확인']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집이 경매로 넘어간다고 들었어요. 저는 보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?
Q(normalized) : 집이 경매로 넘어간다고 들었어요. 저는 임차보증금이 크진 않은데, 일부라도 먼저 받을 수 있나요? 그리고 지역마다 금액이 다르다던데, 어디 기준으로 보는 건가요?
Planner       : use_plan=True | confidence=0.8 | issues=경매 진행 상황, 보증금 반환 가능성, 지역별 차이 확인
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=13 | final=4
Ref articles  : 전세사기피해자 지원 및 주거안정에 관한 특별법 제3조,제6조,제8조,제12조,제14조,제17조,제20조,제78조,제104조,제113조,제114조,제264조,제303조,제496조,제497조 | 주택임대차보호법 제3조,…
-----------------------------------------------------------------------------------

#### 특정시행령은 안가져오지만 공인중개사법으로 우회

In [14]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제14조 제1·2항(주택임대차분쟁조정위원회 + 대상 분쟁) ↔ 시행령 제22조

ask_with_evaluation('집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.')

use_plan: True confidence: 0.6
queries: ['임대차 분쟁 조정 기관 주택임대차보호법', '중개보수 부담 주체 주택임대차', '임차보증금 반환 분쟁 해결 절차', '차임 증액 분쟁 조정 방법', '임대차 계약 분쟁 조정 제도']
need_facts: ['임대차 계약서의 중개보수 조항 내용', '현재 분쟁 중인 차임 및 보증금 금액', '분쟁 발생 시기 및 구체적 사유', '임대인과 협의 진행 상황', '지역별 임대료 상한제 적용 여부', '중개보수 계약서에 명시된 분담 비율']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 집주인이랑 월세랑 보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 집주인이 중개사 수수료를 저보고 다 내라고 하는데요.
Q(normalized) : 임대인이랑 차임랑 임차보증금 문제로 계속 싸우는데, 법원 말고 중간에서 조정해주는 데는 없어요? 그리고 임대인이 중개사 중개보수를 저보고 다 내라고 하는데요.
Planner       : use_plan=True | confidence=0.6 | issues=임대차 분쟁 조정, 중개보수 분쟁, 임차보증금 반환, 차임 분쟁
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=14 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조,제8조,제14조,제20조,제24조
------------------------------------------------------------------------------------------
  - completeness: 0.000 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법

#### 시행령 안가져옴

In [15]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제21조 제1항(조정 신청 관할) ↔ 시행령 제30조·제33조

ask_with_evaluation('제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈은 꼭 내야 하나요?')

use_plan: True confidence: 0.7
queries: ['임대차 계약 해지 조정 신청 절차', '주택임대차 분쟁 조정 관할 법원', '조정 신청 비용 유무', '조정 신청 방식 구두 가능 여부', '임대차 분쟁 지역 관할 기준']
need_facts: ['현재 계약 중인 주택의 정확한 주소', '임대차 계약서 상 기재된 분쟁 해결 조항', '이전 지역과의 계약 관계 유무', '임대인과의 사전 협의 여부', '구체적인 계약 해지 사유', '조정 신청 희망 기관(법원/센터)']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 제가 지금 다른 지역으로 이사 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요? 돈…
Q(normalized) : 제가 지금 다른 지역으로 주택의인도 왔는데요. 조정 신청은 지금 사는 곳에서 하면 되나요, 원래 집 있는 데서 해야 하나요? 그리고 직접 가서 말로만 해도 되나요…
Planner       : use_plan=True | confidence=0.7 | issues=임대차 계약 해지, 조정 신청 절차, 관할 법원, 신청 비용
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=10 | final=4
Ref articles  : 주택임대차보호법 제8조,제14조,제20조,제21조,제22조,제24조
------------------------------------------------------------------------------------------
  - completeness: 0.250 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 

#### 시행령 안가져옴

In [21]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제22조(조정절차 개시 및 송달) ↔ 시행령 제32조

ask_with_evaluation('조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?')

use_plan: True confidence: 0.8
queries: ['조정 신청 후 임대인 연락 절차', '임대인과의 직접 연락 가능성', '임대인의 답변 지연 시 법적 결과', '자동 불리함에 대한 법률 조항', '임대인과의 조정 신청 시 주의사항']
need_facts: ['조정 신청 후 임대인과의 직접 연락 가능 여부 확인', '임대인의 답변 지연 시 구체적인 법적 결과 예시', '자동 불리함 적용 시 임대인의 의무 사항', '조정 신청 시 임대인과의 협의 과정', '임대인의 답변 지연 시 추가 조치 가능 여부', '임대인과의 직접 연락 시 주의해야 할 사항']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 조정 신청하고 나면 집주인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?
Q(normalized) : 조정 신청하고 나면 임대인한테 바로 연락 가나요? 몰래 진행되는 건 아니죠? 그리고 답변 안 하면 자동으로 불리해지나요?
Planner       : use_plan=True | confidence=0.8 | issues=조정 신청 절차, 임대인 연락 여부, 답변 지연 영향, 자동 불리함
Need facts    : 6 item(s)
Retrieval     : k=4 | top_k=4 | pool=15 | final=4
Ref articles  : 주택임대차보호법 제14조,제21조,제22조 | 주택임대차보호법 시행령 제2조,제7조,제8조,제15조 | 민법 제634조
------------------------------------------------------------------------------------------
  - completeness: 0.800 (min=0.99)
  - accuracy    : 1.00

#### 확실한거 안가져옴

In [22]:
%%time
# 모델 : solar-pro2 
# 조항 : 주택임대차보호법 제27조(조정서의 집행력) ↔ 시행령 제34·35조

ask_with_evaluation('조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.')

use_plan: True confidence: 0.7
queries: ['임대인 준수 합의 내용 구체화', '계약증서 내용 확인', '임대인의 계약 위반 사례 검색', '임대차보호법 관련 판례 검색', '민사소송 절차 및 강제집행 관련 정보 검색']
need_facts: ['합의 내용의 구체적인 내용', '임대인의 위반 사항 상세', '계약증서의 서명 및 날짜 확인', '이전 분쟁 사례 유무', '현재 상황의 법적 절차 진행 상황']

🧪 EVAL SUMMARY
------------------------------------------------------------------------------------------
Q(original)   : 조정에서 합의했는데 집주인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 종이인지 판결 같은 건지 헷갈려요.
Q(normalized) : 조정에서 합의했는데 임대인이 또 안 지켜요. 이거 그냥 약속이라 강제로 못 받는 건가요? 이게 그냥 임대차계약증서인지 판결 같은 건지 헷갈려요.
Planner       : use_plan=True | confidence=0.7 | issues=임대인 준수 여부, 계약 유형 확인, 법적 강제력, 증거 필요성
Need facts    : 5 item(s)
Retrieval     : k=4 | top_k=4 | pool=11 | final=4
Ref articles  : 주택임대차보호법 제6조,제7조 | 민법 제625조,제628조
------------------------------------------------------------------------------------------
  - completeness: 0.500 (min=0.99)
  - accuracy    : 1.000 (min=0.90)
  - fail reason : 법령명 누락 문장 2개
  - example(art-only): - **민사조정법 제30조**: 조정조서는 확